<a href="https://colab.research.google.com/github/Ajinkya2023-github/ML_AI_Projects/blob/main/StyleGAN2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## StyleGAN2 with adaptive discriminator augmentation (ADA)

[NVIDIA system requirement for StyleGAN2 ADA](https://github.com/NVlabs/stylegan2-ada)

StyleGAN2 - ADA [Adaptive Discriminator Augmentation] model was created by NVIDIA to avoid the problem of overfitting in the discriminator when the datasets are small.
*   Better results for datasets with less than 30,000 training images.
*   1.6x faster training.
*   1.5x lower GPU memory consumption.




In [1]:
# Clone the StyleGAN2-ADA repository and go inside the directory

!git clone https://github.com/NVlabs/stylegan2-ada.git

fatal: destination path 'stylegan2-ada' already exists and is not an empty directory.


In [2]:
import os
# !mkdir stylegan2_ada
os.path.exists("/content/stylegan2-ada")


True

# CUSTOM DATASET

Code needs the dataset to be in .tfrecords format. Convert our dataset to this format using the StyleGAN2 conversion script.

In [3]:
!pip install kaggle

In [4]:
!mkdir "/content/animals10/"
os.path.exists("/content/animals10")

mkdir: cannot create directory ‘/content/animals10/’: File exists


True

In [5]:
os.path.abspath("kaggle.json")

'/content/kaggle.json'

In [6]:
# Upload kaggle key
import shutil
from google.colab import files

#files.upload()  #kaggle.json already uploaded


{}

In [7]:
import kagglehub
# Move the Kaggle API key to the appropriate directory
# Create the .kaggle directory if not exists
os.makedirs("/root/.config/kaggle", exist_ok=True)

kaggle_path = "/content/kaggle.json"

# Move the kaggle file to .config/kaggle
if (os.path.exists(kaggle_path)):
  shutil.move(kaggle_path, "/root/.config/kaggle/kaggle.json")

else:
  print("Kaggle.json not found!")

# Set permissions to avoid access errors
os.chmod("/root/.config/kaggle/kaggle.json", 600)

#Download the dataset
from kaggle.api.kaggle_api_extended import KaggleApi
api = KaggleApi()
api.authenticate()
kagglehub.dataset_download("alessiocorrado99/animals10")

Kaggle.json not found!


'/kaggle/input/animals10'

# RESIZING THE IMAGES

In [12]:
#os.makedirs("/content/stylegan2-ada/resized")
output_folder = "/content/stylegan2-ada/resized"

In [ ]:

import cv2
import os

# We are using animals10 dataset from kaggle for this project
file_path = '/kaggle/input/animals10/raw-img'


im_size = 1024   # Input image resolution must be a power-of-two otherwise you will get error.
                 #Pixel resolutions that are powers of 2 (512 x 512, 1024 x 1024, 2048 x 2048, etc).



os.makedirs(output_folder, exist_ok = True)

images = []

folder_path = []

for folder in os.listdir(file_path):
    folder_path.append(os.path.join(file_path, folder))

for folder in folder_path:
    if not os.path.isdir(folder):
      continue
    for file in os.listdir(folder):
          img_path = os.path.join(folder, file)
          img = cv2.imread(img_path)  # Load image
          if img is None:                             # Skip if the file is not a valid image
            print(f"❌ Could not read {img_path}, skipping...")
            continue
          img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB (for proper display)


          img = cv2.resize(img, (im_size, im_size))
          #print(f"✅ Processed: {file}, Shape: {img.shape}")

          images.append(img)
          # Save the image in Output Folder
          save_path = os.path.join(output_folder, file.split('.')[0] + "_resized.jpg")
          cv2.imwrite('/content/stylegan2-ada/resized' + str(file) + '_resized.jpg', img)
          #print(f"💾 Saved: {save_path}")


In [4]:
import os
#resized image folder size
resized_img_path = "/content/stylegan2-ada/resized"
print(len(resized_img_path))
os.listdir("/content/stylegan2-ada/resized")

30


[]

### Converting images into .tfrecords

In [ ]:
# Below command will create a multi-resolution .tfrecord file in /custom_dataset/tfrecords_dataset/ folder.
!pip install tqdm


!python dataset_tool.py create_from_images custom_dataset/tfrecords_dataset/ custom_dataset/resized_dataset


In [ ]:
#Training StyleGAN2-ADA

# snap is how often you want to save the model and sample results
# res is what image resolution you want to train on
# augpipe is augmentation pipes, such as 'blit', 'geom', 'color', 'filter', 'noise', 'cutout' or combination of these



!python train.py --outdir ./results --snap=10 --data=custom_dataset/tfrecords_dataset --augpipe=bgcfnc --res=512


In [ ]:
# There are many other arguments that you can modify, feel free to check the train.py code to learn more about the arguments.
# Once you run the command, it will start training and periodically save the result and the model file (.pkl) based on the snap arguments
# that you provided (In this case, every 10kimg). Once you think that the result is good enough or the FID starts to plateau, you can stop training and use the last saved .pkl file.

# Once you have the model file you can generate images using this command.

#!python generate.py --outdir=out --trunc=0.5 --seeds=600-605 --network={path_to_pkl_model_file}

Pretrained Model Training

https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada/pretrained/

In [ ]:
# Generate class conditional CIFAR-10 images (Fig.17 left, Car)
#!python generate.py --outdir=out --trunc=1 --seeds=0-35 --class=1 --network=https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada/pretrained/cifar10.pkl

!python generate.py --outdir=out1 --trunc=1 --seeds=0-10 --class=1 --network=https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada/pretrained/cifar10.pkl